In [1]:
import pprint
import re

In [2]:
# 国土地理院が公開したファイルを読み込む
import json
json_open = open('20200903.geojson', 'r', encoding='utf-8')
geojson = json.load(json_open)

In [3]:
pprint.pprint(geojson['features'][0])

{'geometry': {'coordinates': [142.390215, 43.681137], 'type': 'Point'},
 'properties': {'ID': '01204-001',
                '伝承内容': '昭和22年（1947）8月15日～16日の豪雨は、大正4年(1915) '
                        'に次ぐ約30年ぶりの大洪水となり、尊い命が犠牲になったほか家屋の浸水、田畑の流出等の被害が発生した。',
                '建立年': '1958',
                '所在地': '北海道旭川市西神楽南１条3丁目',
                '災害名': '昭和22年8月洪水（1947年8月15日～16日）',
                '災害種別': '洪水',
                '碑名': '美瑛川治水竣功記念'},
 'type': 'Feature'}


In [4]:
# 先にdescriptionリンク用のHTMLテーブルを作成する
table = '<table><tr><th>ID</th><th>碑名</th><th>建立年</th><th>所在地</th><th>災害名</th><th>災害種別</th><th>伝承内容</th></tr>'
features = geojson['features']

for feature in features:
    p = feature['properties']
    row = '<tr><td>' + p['ID'] + '</td><td>' + p['碑名'] + '</td><td>' + p['建立年'] + '</td><td>' + p['所在地'] + '</td><td>' + p['災害名'] + '</td><td>' + p['災害種別'] + '</td><td id="id' + p['ID'] + '">' + p['伝承内容'] + '</td></tr>\n'
    table += row

table += '</table>'
print(table)


<table><tr><th>ID</th><th>碑名</th><th>建立年</th><th>所在地</th><th>災害名</th><th>災害種別</th><th>伝承内容</th></tr><tr><td>01204-001</td><td>美瑛川治水竣功記念</td><td>1958</td><td>北海道旭川市西神楽南１条3丁目</td><td>昭和22年8月洪水（1947年8月15日～16日）</td><td>洪水</td><td id="id01204-001">昭和22年（1947）8月15日～16日の豪雨は、大正4年(1915) に次ぐ約30年ぶりの大洪水となり、尊い命が犠牲になったほか家屋の浸水、田畑の流出等の被害が発生した。</td></tr>
<tr><td>01208-001</td><td>樹霊供養之塔</td><td>1959</td><td>北海道北見市留辺蘂町富士見</td><td>洞爺丸台風（1954年9月26日）</td><td>その他</td><td id="id01208-001">昭和29年（1954）9月26日、北海道に襲来した台風15号（洞爺丸台風）は、一夜にして道内の広い範囲で立木をなぎ倒し、森林に大規模な被害を与えた。中でも層雲峡地帯北見富士山麓の被害は甚だしく、世界林業史上例をみない風倒木を出現させた。</td></tr>
<tr><td>01333-001</td><td>復興記念碑</td><td>1976</td><td>北海道上磯郡知内町字小谷石</td><td>昭和48年集中豪雨（1973年9月24日）</td><td>土砂災害</td><td id="id01333-001">昭和48年(1973)9月23日から25日にかけて、強い集中豪雨が道南地方を襲い、特に24日13時からの1時間雨量は133mmを記録した。この大雨による土石流とがけ崩れ等で小谷石地区全戸数の40%にあたる100戸が全壊又は流失し、死者4名、行方不明者4名の被害が生じた。碑には当時中ノ沢上流から流れてきた自然石が使用されている。</td></tr>
<tr><td>01367-001</td><td>時空翔</td><td>1998</td><td>北海道奥尻郡奥尻町字青苗49</td><td>北海道南西沖地震(1993年7月

In [5]:
# データの中身をOSM形式に変換
features = geojson['features']

for feature in features:
    properties = feature['properties']
    
    # 基本タグ
    tags = {'historic': 'memorial', 'memorial': 'hazard_memorial'}
    
    # 建立年が特異値の場合に対処
    start_date = properties['建立年']
    if '？' in start_date:
        start_date = '~' + re.search('(\d{4})',start_date)[1]
    elif '不明' == start_date:
        start_date = ''
    elif ' ' in start_date:
        start_date = re.search('(\d{4})',start_date)[1]
    elif '江戸時代' == start_date:
        start_date = '1603..1868'
    if start_date:
        tags['start_date'] = start_date
    
    tags['addr:full'] = properties['所在地']
    tags['hazard:name'] = properties['災害名']
    
    # 碑名のかっこ書き部分を削除
    name = re.sub('[\(（]([^\)）]*)[\)）]', '',properties['碑名'])
    tags['name'] = name
    
    # 災害種別を置き換え
    # 洪水(flood);土砂災害(landslide);火山災害(volcano);高潮(storm_surge);地震(earthquake)
    hazard_type = properties['災害種別']
    hazard_type = re.sub('洪水', 'flood',hazard_type)
    hazard_type = re.sub('土砂災害', 'landslide',hazard_type)
    hazard_type = re.sub('火山災害', 'volcano',hazard_type)
    hazard_type = re.sub('高潮', 'storm_surge',hazard_type)
    hazard_type = re.sub('地震', 'earthquake',hazard_type)
    hazard_type = re.sub('津波', 'tsunami',hazard_type)
    hazard_type = re.sub('・', ';',hazard_type)
    hazard_type = re.sub('その他', '',hazard_type)
    if hazard_type:
        tags['hazard:type'] = hazard_type
    
    # descriptionは255文字を超える場合があるので、外部サイトへのリンクにする。
    tags['description:url'] = 'https://tankaru.github.io/Japan-GSI-Import/data.html#id' + properties['ID']
    
    feature['properties'] = tags




In [6]:
pprint.pprint(features[0])

{'geometry': {'coordinates': [142.390215, 43.681137], 'type': 'Point'},
 'properties': {'addr:full': '北海道旭川市西神楽南１条3丁目',
                'description:url': 'https://tankaru.github.io/Japan-GSI-Import/data.html#id01204-001',
                'hazard:name': '昭和22年8月洪水（1947年8月15日～16日）',
                'hazard:type': 'flood',
                'historic': 'memorial',
                'memorial': 'hazard_memorial',
                'name': '美瑛川治水竣功記念',
                'start_date': '1958'},
 'type': 'Feature'}


In [7]:
import json
with open('natural_hazard_memorial_import.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, indent=4, ensure_ascii=False)